牌照号码训练

In [187]:
import os
import os.path
import shutil
import cv2
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [188]:
DIGITS='01234567890'
LETTER='ABCDEFGHJKLMNPQRSTUVWXYZ'
PROV='川鄂赣甘贵桂黑沪冀津京吉辽鲁蒙闽宁青琼陕苏晋皖湘新豫渝粤云藏浙'
CHAR=DIGITS+LETTER
CLASSES=['0','1','2','3','4','5','6','7','8','9','0','A','B','C','D','E','F','G','H','J','K','L','M','N','P','Q','R','S','T','U','V','W','X','Y','Z',
         'zh_cuan','zh_e','zh_gan','zh_gan1','zh_gui','zh_gui1','zh_hei','zh_hu','zh_ji','zh_jin','zh_jing','zh_jl','zh_liao','zh_lu','zh_meng',
        'zh_min','zh_ning','zh_qing','zh_qiong','zh_shan','zh_su','zh_sx','zh_wan','zh_xiang','zh_xin','zh_yu','zh_yu1','zh_yue','zh_yun',
        'zh_zang','zh_zhe']

In [189]:
def dirname_to_class(cname):
    return CLASSES.index(cname)

def find_classes(dir):
    classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx

In [194]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.norm2d=nn.BatchNorm2d(1)
        self.conv1 = nn.Conv2d(1, 10, kernel_size=3,padding=1)   
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3,padding=1) 
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(15*15*20, 1000)
        self.fc2 = nn.Linear(1000, 65)

    def forward(self, x):
        x = self.norm2d(x)
        x = F.relu(F.max_pool2d(self.conv1(x), 2))                                #60*60
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))   #30*30
        x = x.view(-1, 15*15*20)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)    
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 100)
        self.fc2 = nn.Linear(100, 34)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))                              #28*28
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2)) #12*12
        x = x.view(-1, 4*4*20)                                                                #8*8
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)    

In [195]:
def gray_loader(path):
    im=cv2.imread(path,cv2.CV_LOAD_IMAGE_GRAYSCALE)
    return  im

In [196]:
TRAINDIR='/home/wang/git/nppic/ann/train'
VALIDATEDIR='/home/wang/git/nppic/ann/val'
#73.9896738689 13392.8949973
data_transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((73.9896 ,), (13392.8949,)),
                             ])
trainset = datasets.ImageFolder(root=TRAINDIR, 
                             transform=data_transform,loader=gray_loader)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=10,shuffle=True, num_workers=1)
net=Net()
criterion = nn.CrossEntropyLoss() # use a Classification Cross-Entropy loss
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [197]:
for epoch in range(10): # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs=torch.unsqueeze(inputs,1)
        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)
        #inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda()) #cuda
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.data[0]
        if i % 500 == 0: 
            #print('[%d, %5d] loss: %.3f' % (epoch+1, i, running_loss / 2000))
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(data), len(trainloader.dataset),
                100. * i / len(trainloader), loss.data[0]))
    #print ('Train Epoch: {} total running_loss: {}'.format(epoch,running_loss))
    running_loss = 0.0
print('Finished Training')

Train Epoch: 0 [0/12565 (0%)]	Loss: 3.526878
Train Epoch: 0 [1000/12565 (40%)]	Loss: 2.805736
Train Epoch: 0 [2000/12565 (80%)]	Loss: 0.769635
Train Epoch: 1 [0/12565 (0%)]	Loss: 1.507317
Train Epoch: 1 [1000/12565 (40%)]	Loss: 0.669195
Train Epoch: 1 [2000/12565 (80%)]	Loss: 0.303886
Train Epoch: 2 [0/12565 (0%)]	Loss: 1.098296
Train Epoch: 2 [1000/12565 (40%)]	Loss: 0.276625
Train Epoch: 2 [2000/12565 (80%)]	Loss: 0.438980
Train Epoch: 3 [0/12565 (0%)]	Loss: 0.276811
Train Epoch: 3 [1000/12565 (40%)]	Loss: 1.073763
Train Epoch: 3 [2000/12565 (80%)]	Loss: 0.444083
Train Epoch: 4 [0/12565 (0%)]	Loss: 0.231498
Train Epoch: 4 [1000/12565 (40%)]	Loss: 0.198737
Train Epoch: 4 [2000/12565 (80%)]	Loss: 0.537617
Train Epoch: 5 [0/12565 (0%)]	Loss: 0.076261
Train Epoch: 5 [1000/12565 (40%)]	Loss: 0.098568
Train Epoch: 5 [2000/12565 (80%)]	Loss: 0.117362
Train Epoch: 6 [0/12565 (0%)]	Loss: 0.444892
Train Epoch: 6 [1000/12565 (40%)]	Loss: 0.070944
Train Epoch: 6 [2000/12565 (80%)]	Loss: 0.065471

In [198]:
torch.save(net.state_dict(),'/home/wang/git/anpr/coder.net.weight')

以下是验证/测试

In [199]:
model=Net()
model.load_state_dict(torch.load('/home/wang/git/anpr/coder.net.weight'))

In [200]:
#SRCDIR='/home/wang/git/nppic/ann/val'
#cat,idx=find_classes(SRCDIR)
valset = datasets.ImageFolder(root=VALIDATEDIR,
                             transform=data_transform,loader=gray_loader)
valoader = torch.utils.data.DataLoader(valset, batch_size=10, shuffle=True, num_workers=1)

cnt=0
correct=0
for data in valoader:
    images, labels = data
    images=torch.unsqueeze(images,1)
    outputs = model(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    cnt+=len(c)
    correct+=torch.sum(c)

print "cnt: {}, correct: {}, ratio: {:.2f}%".format(cnt,correct,correct/cnt*100.)
print 'over'

cnt: 610, correct: 571, ratio: 0.00%
over


以下是工具函数



In [179]:
#easypy的jpg原图，二值化后，写入bmp，原jpg图像删除
#trian，为二值化后的图像. 二值bmp图像写入文件，再读取
SRCDIR='/home/wang/git/nppic/ann/train'
for root,dirnames,filenames in os.walk(SRCDIR):
    if len(filenames)==0:
        continue
    for i,fname in enumerate(filenames):
        source=os.path.join(root,fname)
        im=cv2.imread(source,cv2.CV_LOAD_IMAGE_GRAYSCALE)
        im=cv2.resize(im,(28,28))
        _,timg  = cv2.threshold(im, 100, 255, cv2.THRESH_BINARY)                
        #os.unlink(source)
        a=source.split('.')
        source='{}.bmp'.format(a[0])
        #cv2.imwrite(source,timg)
print('over')        

over


In [127]:
#bmp图像扩大三倍，20*20-->60*60
SRCDIR='/home/wang/git/nppic/ann/train'
#SRCDIR='/home/wang/git/nppic/ann/val'
#SRCDIR='/home/wang/git/nppic/ann/mean'
for root,dirnames,filenames in os.walk(SRCDIR):
    if len(filenames)==0:
        continue
    for i,fname in enumerate(filenames):
        source=os.path.join(root,fname)
        im=cv2.imread(source,cv2.CV_LOAD_IMAGE_GRAYSCALE)
        im=cv2.resize(im,(60,60))
        #cv2.imwrite(source,im)
        
print('over')    

over


In [180]:
#train 目录所有复制到mean目录
SRCDIR='/home/wang/git/nppic/ann/train'
DSTDIR='/home/wang/git/nppic/ann/mean/'
for root,dirnames,filenames in os.walk(SRCDIR):
    if len(filenames)==0:
        continue
    for i,fname in enumerate(filenames):
        source=os.path.join(root,fname)
        #shutil.copy(source,DSTDIR)

In [182]:
#计算mean目录下文件的mean和varience
DSTDIR='/home/wang/git/nppic/ann/mean'
for root,dirnames,filenames in os.walk(DSTDIR):
    ret=np.ndarray((len(filenames),28,28),np.uint8)
    for i,fname in enumerate(filenames):
        filename=os.path.join(root,fname)
        img=cv2.imread(filename,cv2.CV_LOAD_IMAGE_GRAYSCALE)
        ret[i]=img
#ret=ret
mean=np.mean(ret) 
var=np.var(ret)
print mean,var
#mean:std:72.2949871181 114.928919566    
#mean:var:72.2949871181 13208.6565527   0.283509753404 0.203131973129

73.9896738689 13392.8949973


In [181]:
#从train各子目录移动若干文件到val，同时建立对应的子目录
SRCDIR='/home/wang/git/nppic/ann/train'
DSTDIR='/home/wang/git/nppic/ann/val'
for root,dirnames,filenames in os.walk(SRCDIR):
    if len(filenames)==0:
        continue
    d=os.path.join(DSTDIR,os.path.split(root)[1])
    #os.mkdir(d)
    num=len(filenames)//20
    if num<2:
        continue
    for i in range(num-1):
        srcfilename=os.path.join(root,filenames[i])
        #shutil.move(srcfilename,d)
print('over')        

over


In [ ]:
class NUMPSET(torch.utils.data.Dataset):
    picroot='np'

    def __getitem__(self,index):
        label,img=self.labels[index], self.dataset[index]
        if self.data_transform is not None:
            img=self.data_transform(img)
        #return (img,dirname_to_class(label))
        return img,label

    def __len__(self):
        return self.len

    def __init__(self,root,data_transform=None):
        self.picroot=root
        self.data_transform=data_transform

        if not os.path.exists(self.picroot):
            raise RuntimeError('{} doesnot exists'.format(self.picroot))
        for root,dnames,filenames in os.walk(self.picroot):
            imgs=[]
            labels=[]
            for filename in filenames:
                picfilename=os.path.join(self.picroot,filename)  #file name:
                im=cv2.imread(picfilename,cv2.IMREAD_GRAYSCALE)
                _,timg  = cv2.threshold(im, 100, 255, cv2.THRESH_BINARY)                
                imgs.append(cv2.resize(timg,(20,20)))  #二值化
                m=filename.split('_')  #filename style: x_yyyy.jpg  x is directory and class name
                labels.append(m[0])
            self.dataset=imgs
            self.labels=labels
            self.len=len(filenames)